<div style="border:solid Blue 2px; padding: 40px">
<h1> Прогноз оттока клиентов банка </h1>

- Из банка стали уходить клиенты. По немногу каждый месяц. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
    - Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. в распоряжении исторические данные о поведении клиентов и расторжении договоров с банком. 

    ***Задача*** построить модель для задачи классификации с предельно большим значением F1-меры и нужно довести метрику до 0.59
    Дополнительно измеримерять AUC-ROC. Подобрать гиперпараметры

<div style="border:solid Green 2px; padding: 40px">
<h1> Признаки </h1>
    
 - RowNumber — индекс строки в данных
 - CustomerId — уникальный идентификатор клиента
 - Surname — фамилия
 - CreditScore — кредитный рейтинг
 - Geography — страна проживания
 - Gender — пол
 - Age — возраст
 - Tenure — сколько лет человек является клиентом банка
 - Balance — баланс на счёте
 - NumOfProducts — количество продуктов банка, используемых клиентом
 - HasCrCard — наличие кредитной карты
 - IsActiveMember — активность клиента
 - EstimatedSalary — предполагаемая зарплата
 
  Целевой признак 
 - Exited — факт ухода клиента
 
 Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.shape

(10000, 14)

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [7]:
data.Tenure = data.Tenure.fillna(data.Tenure.median())

In [8]:
data.Tenure.unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0.])

<div style="border:solid Blue 2px; padding: 40px">
<h3> Ревью </h3>   
 
   - Открыли и просмотрели данные. Типы данных корректные, но имеются пропуски в колонке Tenure 
   - Пропуски в этой колонке можно заполнить заглушками (плейсхолдер), медианой (сгруппированной по возрасту, полу или стране) или медианным значением по колонке - выбрали последнее.

### Подготовка тренировочной, валидационной и тестовой выборок

In [9]:
target = data['Exited']
features = data.drop(['Exited', 'Surname', 'RowNumber', 'CustomerId'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345) 

features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.5, random_state=12345)

features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

scaler = StandardScaler()
scaler.fit(features_train) 
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid) 
features_test = scaler.transform(features_test)

In [10]:
features_train = pd.DataFrame(features_train, index=target_train.index)

In [11]:
features_valid = pd.DataFrame(features_valid, index=target_valid.index)

In [12]:
features_test = pd.DataFrame(features_test, index=target_test.index)

    Провели предподготовку данных. Проверяем размеры выборок

In [13]:
features_train.shape

(7500, 11)

In [14]:
target_train.shape

(7500,)

## Исследование задачи

<div style="border:solid Blue 2px; padding: 40px">

     Изучим и обучим модели из библиотеки sklearn:

**2.1 LogisticRegression**

In [15]:
%%time

model_1 = LogisticRegression(penalty='l2', random_state=12345)
model_1.fit(features_train, target_train)
predictions = model_1.predict(features_valid)


print("F1 score", f1_score(target_valid, predictions))
print('ROC AUC LR:', roc_auc_score(target_valid, model_1.predict_proba(features_valid)[:, 1]))

F1 score 0.2960893854748603
ROC AUC LR: 0.7668473424480219
CPU times: user 47.3 ms, sys: 70 ms, total: 117 ms
Wall time: 22 ms


**2.2 DecisionTreeClassifier**

<div style="border:solid Blue 2px; padding: 40px">

    Для DecisionTree, RandomForest лучше всего подбирать параметры путем автоподбора с классом GridSearchCV c кросс-валидацией

In [18]:
%%time

clf_dt = DecisionTreeClassifier()
parametrs = {'criterion': ['entropy'],
             'max_depth': range(1, 10, 2),
             'min_samples_split': range(2, 30, 2),
             'min_samples_leaf': range(1, 20, 2)}

search_param = GridSearchCV(clf_dt, parametrs, cv=5) 
search_param.fit(features_train, target_train)

best_tree = search_param.best_estimator_

CPU times: user 55.3 s, sys: 89 ms, total: 55.4 s
Wall time: 55.5 s


In [19]:
best_tree

DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=9,
                       min_samples_split=26)

In [16]:
model_2 = DecisionTreeClassifier(
    criterion='entropy', 
    max_depth=7, 
    min_samples_split=26,
    min_samples_leaf=9,
    random_state=12345    
)
model_2.fit(features_train, target_train)
predictions = model_2.predict(features_valid)

print("F1", f1_score(target_valid, predictions))
print('ROC AUC Dtc:', roc_auc_score(target_valid, model_2.predict_proba(features_valid)[:, 1]))

F1 0.5621890547263682
ROC AUC Dtc: 0.8558561682095376


    
    F1 мера равняется 0.56 а требуется не менее 0.59. 
    Перспективно, но смотрим дальше.

**2.3 RandomForest**

In [49]:
%%time

clf_rf = RandomForestClassifier()
parametrs = { 
    'n_estimators': range(10, 110, 10),
    'criterion': ['gini'],
    'max_depth': range(2, 10, 2),
    'min_samples_split': range(2, 10, 2),
    'min_samples_leaf': range(2, 10, 2)
}

search_param = GridSearchCV(clf_rf, parametrs, cv=5) 
search_param.fit(features_train, target_train)

best_forest = search_param.best_estimator_

CPU times: user 10min 46s, sys: 4.94 s, total: 10min 51s
Wall time: 10min 51s


In [50]:
best_forest

RandomForestClassifier(max_depth=8, min_samples_leaf=8, min_samples_split=6,
                       n_estimators=50)

In [17]:
%%time

model_3 = RandomForestClassifier(
    criterion='gini', 
    n_estimators=50,
    max_depth=8, 
    min_samples_leaf=8,
    min_samples_split=6,
    random_state=12345    
)
model_3.fit(features_train, target_train)
predictions = model_3.predict(features_valid)

print("F1", f1_score(target_valid, predictions))
print('ROC AUC Rf:', roc_auc_score(target_valid, model_3.predict_proba(features_valid)[:, 1]))

F1 0.585
ROC AUC Rf: 0.8681028272485274
CPU times: user 312 ms, sys: 2.99 ms, total: 315 ms
Wall time: 315 ms


<div style="border:solid Blue 2px; padding: 40px">
    
        AucRoc метрика у Дерева и Леса сравнялась, а F-мера разлиается - судя по ней, вторая модель качественнее. Самой перспективной в предсказывании целевого признака (уход клиентов банка) показала себя модель RandomForestClassifier

## Борьба с дисбалансом

<div style="border:solid Blue 2px; padding: 40px">
    
    Рассмотрим способы борьбы с дисбалансом классов путем понижения, увеличения и балансирования.

### Функция понижение классов downsample

In [18]:
def downsample(features_train, target_train, fraction):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)


<div style="border:solid Blue 2px; padding: 40px">
    
    Борьба с дисбалансом путем увеличении выборки и перемешивания upsampling и shuffle. Создадим функцию для увеличения редких классов

### Функция увеличение классов upsample

In [19]:
def upsample(features_train, target_train, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 12)

<div style="border:solid Blue 2px; padding: 40px">
    
    Протестируем увеличенные и уменьшенные выборки на трех моделях.

***LogisticRegression***

In [21]:
model_lr_balance = LogisticRegression(random_state=12345, solver='liblinear')
model_lr_balance.fit(features_upsampled, target_upsampled)
predicted_valid_lr = model_lr_balance.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_lr))
print('ROC AUC LR_1:', roc_auc_score(target_valid, model_lr_balance.predict_proba(features_valid)[:, 1]))

F1: 0.4039247751430907
ROC AUC LR_1: 0.7725218717856853


In [23]:
model_lr_balance.fit(features_downsampled, target_downsampled)
predicted_valid_lr = model_lr_balance.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_lr))
print('ROC AUC LR_1:', roc_auc_score(target_valid, model_lr_balance.predict_proba(features_valid)[:, 1]))

F1: 0.4197530864197531
ROC AUC LR_1: 0.7708923226276191


In [24]:
model_lr_balance = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_lr_balance.fit(features_train, target_train)
predicted_valid_lr = model_lr_balance.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_lr))
print('ROC AUC LR_1:', roc_auc_score(target_valid, model_lr_balance.predict_proba(features_valid)[:, 1]))

F1: 0.5112582781456954
ROC AUC LR_1: 0.7706727379892981


<div style="border:solid Blue 2px; padding: 40px">
    
- Обучение Логистической модели на увеличенной выборке features_upsampled, при валидировании, дает 40% правильных ответов
- На уменьшенной features_downsampled 41%
- Баланс классов прописанный в атрибуте class_weight='balanced' значительно улучшает предсказание по сравнению с дредыдущими способами борьбы с дисбалансом и повышает качество модели до 51%

***DecisionTreeClassifier***

In [25]:
model_dt_balance = DecisionTreeClassifier(
    criterion='entropy', 
    max_depth=7, 
    min_samples_split=26,
    min_samples_leaf=9,    
)
model_dt_balance.fit(features_upsampled, target_upsampled)
predicted_valid_dt = model_dt_balance.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_dt))
print('ROC AUC DTC:', roc_auc_score(target_valid, model_dt_balance.predict_proba(features_valid)[:, 1]))

F1: 0.482
ROC AUC DTC: 0.8577149329111142


После увеличения выборки f1 score cнизился


In [26]:
model_dt_balance.fit(features_downsampled, target_downsampled)
predicted_valid_dt = model_dt_balance.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_dt))
print('ROC AUC DTC:', roc_auc_score(target_valid, model_dt_balance.predict_proba(features_valid)[:, 1]))

F1: 0.5044843049327354
ROC AUC DTC: 0.8108297602675081


После уменишения выборки f1 score стал лучше

In [27]:
model_dt_balance = DecisionTreeClassifier(
    criterion='entropy', 
    max_depth=7, 
    min_samples_split=26,
    min_samples_leaf=9,
    class_weight='balanced',
    random_state=12345    
)
model_dt_balance.fit(features_downsampled, target_downsampled)
predicted_valid_dt = model_dt_balance.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_dt))
print('ROC AUC DTC:', roc_auc_score(target_valid, model_dt_balance.predict_proba(features_valid)[:, 1]))

F1: 0.5437665782493369
ROC AUC DTC: 0.8109742238453508


<div style="border:solid Blue 2px; padding: 40px">     

- Обучение Дерева решений на увеличенной выборке features_upsampled, при валидировании, дает 48% правильных ответов
- На уменьшенной features_downsampled 50%
- Баланс классов прописанный в атрибуте class_weight='balanced' и с обучением на downsample выборке значительно улучшает предсказание по сравнению с предыдущими способами борьбы с дисбалансом и повышает качество модели до 54%

***RandomForestClassifier***

In [28]:
model_rf_balanced = RandomForestClassifier(
    criterion='gini', 
    n_estimators=50,
    max_depth=8, 
    min_samples_leaf=8,
    min_samples_split=6,
    random_state=12345      
)
model_rf_balanced.fit(features_upsampled, target_upsampled)
predicted_valid_rf = model_rf_balanced.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_rf))
print('ROC AUC RF:', roc_auc_score(target_valid, model_rf_balanced.predict_proba(features_valid)[:, 1]))

F1: 0.4883955600403633
ROC AUC RF: 0.8670241658673015


In [29]:
model_rf_balanced.fit(features_downsampled, target_downsampled)
predicted_valid_rf = model_rf_balanced.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_rf))
print('ROC AUC RF:', roc_auc_score(target_valid, model_rf_balanced.predict_proba(features_valid)[:, 1]))

F1: 0.47058823529411764
ROC AUC RF: 0.8625014927903044


In [30]:
model_rf_best = RandomForestClassifier(
    criterion='gini',  
    n_estimators=50,
    max_depth=8, 
    min_samples_leaf=8,
    min_samples_split=6,  
    class_weight='balanced',
    random_state=12345    
)
model_rf_best.fit(features_upsampled, target_upsampled)
predicted_valid_rf = model_rf_best.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_rf))
print('ROC AUC RF:', roc_auc_score(target_valid, model_rf_best.predict_proba(features_valid)[:, 1]))

F1: 0.639871382636656
ROC AUC RF: 0.8703487543387228


<div style="border:solid Blue 2px; padding: 40px">    
    
- Обучение модели Случайного леса на увеличенной выборке features_upsampled, при валидировании, дает 48% правильных ответов, как и у дерева
- На уменьшенной features_downsampled 47%
- Баланс классов прописанный в атрибуте class_weight='balanced' и обучение на upsampled выборке значительно улучшает предсказание по сравнению с предыдущими способами борьбы с дисбалансом и повышает качество модели до 63%

## Тестирование модели

***Тестируем RandomForest***

In [31]:
predict_test = model_rf_best.predict(features_test)

In [32]:
print("F1:", f1_score(target_test, predict_test))
print('ROC AUC RF:', roc_auc_score(target_test, model_rf_best.predict_proba(features_test)[:, 1]))

F1: 0.6341463414634146
ROC AUC RF: 0.8589445747624203


<div style="border:solid Blue 2px; padding: 40px">
<h1>  Итоговый комментарий </h1>

- Получилась хорошая модель с требуемым значением, цель довести метрику до 0.59 - получилось 0.63, а также метрика roc_auc 0.85- высокий показатель.

- Выполнена предподготовка данных, заполнены пропуски в данных tenure

- Выполнена разбивка данных на тренировочную, валидационную и тестовую выборки, а далее преобразование данных (что-бы избежать утечки).
    
- Обучено три модели LogisticRegression, DecisionTreeClassifier, RandomForestClassifier   
    
- Применение класса GridSearchCV для подбора оптимальных параметров модели
    
- Изучены метрики F1 score и auc_roc в сравнении
- Реализовано три метода борьбы с дисбалансом downsampling, class_weight и upsampling
    
- В итоге лучшей моделью стал ***RandomForest***, на нем была обучена тестовая выборка где финальная метрика f1 была равна 0.63, а auc_roc 0.85.